In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from google.cloud.firestore_v1.document import DocumentReference


from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


def load_json_file(filename):
    with open(filename) as fp:
        content = json.load(fp)
    return content


In [4]:

def process_reference(reference: DocumentReference):
    return reference.path

def process_array(array: list):
    return [process_reference(item) if type(item) == DocumentReference else item for item in array]

def process_dict(input_dict):
    output_dict = {}
    for key, value in input_dict.items():
        if type(value) == DocumentReference:
            output_dict[key] = process_reference(value)
        elif isinstance(value, dict):
            output_dict[key] = process_dict(value)
        elif isinstance(value, list):
            output_dict[key] = process_array(value)
        elif isinstance(value, datetime):
            output_dict[key] = value.isoformat()
        else:
            output_dict[key] = value
    return output_dict

## Carregamentos

In [12]:
pesquisa = db.collection("lotes").where(filter=FieldFilter("tipo", "==", "Carregamento"))

CARREGAMENTOS = {}
for document in pesquisa.stream():
    data = document.to_dict() 
    CARREGAMENTOS[document.id] = process_dict(data)

In [13]:
pesquisa = db.collection("lotes").where(filter=FieldFilter("tipo", "==", "Descarregamento"))

DESCARREGAMENTOS = {}
for document in pesquisa.stream():
    data = document.to_dict() 
    DESCARREGAMENTOS[document.id] = process_dict(data)

In [15]:
import json 

with open("../json0704/carregamentos.json", "w") as fp:
    json.dump(CARREGAMENTOS, fp, indent=4, ensure_ascii=False)

with open("../json0704/descarregamentos.json", "w") as fp:
    json.dump(DESCARREGAMENTOS, fp, indent=4, ensure_ascii=False)

## Gravação em lote

In [16]:
from datetime import datetime

In [28]:
for key, value in DESCARREGAMENTOS.items(): 
    data_criado = datetime.fromisoformat(value["data_criado"])
    volumes = value["volumes"]
    responsavel = db.document(value["responsavel"])
    sala_ref = db.document(value["sala"]) 
    lote_ref = db.document(f"lotes/{key}")
    """
    atualizar: volume.localizacao_atual: sala_ref
    movimentos/${volume_id}
    descarregado: { 
                    data_hora: lote.data_criado,
                    resposavel: lote.responsavel,
                    sala: sala_ref,
                    lote: lote_ref
                }
    """
    volumes_refs = map(lambda volume: db.document(volume), volumes)
    batch = db.batch()
    for volume_ref in volumes_refs:
        volume_id = volume_ref.id
        batch.update(volume_ref, {"localizacao_atual": sala_ref})
        movimento_ref = db.document(f"movimentos/{volume_id}")
        if movimento_ref.get().exists:
            batch.update(movimento_ref, {"descarregado": {
                "data_hora": data_criado,
                "responsavel": responsavel,
                "sala": sala_ref,
                "lote": lote_ref
            }})
        else:
            batch.set(movimento_ref, {"descarregado": {
            "data_hora": data_criado,
            "responsavel": responsavel,
            "sala": sala_ref,
            "lote": lote_ref
        }}) 
    batch.commit() 

In [29]:
for key, value in CARREGAMENTOS.items(): 
    data_criado = datetime.fromisoformat(value["data_criado"])
    volumes = value["volumes"]
    responsavel = db.document(value["responsavel"]) 
    lote_ref = db.document(f"lotes/{key}")
    carregamento_ref = db.document(value["carregamento"])
    """
    atualizar: volume.localizacao_atual: sala_ref
    movimentos/${volume_id}
    descarregado: { 
                    data_hora: lote.data_criado,
                    resposavel: lote.responsavel,
                    sala: sala_ref,
                    lote: lote_ref
                }
    """
    volumes_refs = map(lambda volume: db.document(volume), volumes)
    batch = db.batch()
    for volume_ref in volumes_refs:
        volume_id = volume_ref.id 
        movimento_ref = db.document(f"movimentos/{volume_id}")
        if movimento_ref.get().exists:
            batch.update(movimento_ref, {"carregado": {
                "data_hora": data_criado,
                "responsavel": responsavel, 
                "lote": lote_ref,
                "carregamento": carregamento_ref
            }})
        else:
            batch.set(movimento_ref, {"carregado": {
            "data_hora": data_criado,
            "responsavel": responsavel,
            "lote": lote_ref,
            "carregamento": carregamento_ref
        }}) 
    batch.commit() 